# A Simple System Design for Forecasting Time Series Stock Data

# Imports

In [68]:
import numpy as np
import pandas as pd
import yfinance as yf

from typing import List
from dataclasses import dataclass

# Data Processor - Load the stock data

- Retrieve the stock data with `data = yf.download(symbol, start="YYYY-MM-DD", end="YYYY-MM-DD")`
- Specific what column we're seeking to explore

In [70]:
@dataclass
class YFinanceDataProcessor:
    """Handles fetching and returning data. """
    ticker_symbol: str
    start_date: str
    end_date: str
    
    def __post_init__(self):
        """Operations that are performed after the initialization step."""
        self.data = yf.download(self.ticker_symbol, start=self.start_date, end=self.end_date)
    
    def filter_by_col(self, col_name):
        """Returns a filtered dataframe with all the rows."""
        univariate_ts_df = self.data.loc[:, [col_name]]
        return univariate_ts_df

In [67]:
# Set the start and end dates for the data
# symbol = "VOO"
symbol = "BRK-A"
start_date = "2022-01-01"
end_date = "2023-06-25"


data_processor = YFinanceDataProcessor(symbol, start_date, end_date)
# data_processor.data
data = data_processor.filter_by_col("Open")
data

[*********************100%***********************]  1 of 1 completed


,Open
Date,
2022-01-03,452005.0
2022-01-04,458675.0
2022-01-05,468500.0
2022-01-06,470900.0
2022-01-07,474696.0
...,...
2023-06-12,511725.0
2023-06-13,507627.0
2023-06-14,519500.0
